In [ ]:
!pip install imagecodecs-lite
!pip install imagecodecs
!pip uninstall tiffile
import imagecodecs

from skimage.io import imread, imshow
import numpy as np
import matplotlib.pyplot as plt
import csv
from random import shuffle


from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.optimizers import Adam

import tensorflow as tf


def load_data(path):
    info = []
    images_list = []
    images_class = []
    print('Loading ', path, 'in memory...')
    with open(path, newline='') as csv_file:
        images_reader = csv.reader(csv_file)
        next(images_reader)
        for row in images_reader:
            info.append(row)
    shuffle(info)
    i = 0
    for elem in info:
        i += 1 
        if i%100 == 0 : print(i)
        images_list.append(imread('/content/drive/MyDrive/Colab Notebooks/Train&Validation/' + elem[0])[55:455, 55:455].reshape(400, 400, 1))
        images_class.append(elem[1] == 'covid')
    return np.array(images_list), np.array(images_class)


def create_model():
    model = Sequential()
    model.add(Conv2D(32, 3, padding="same", activation="relu", input_shape=(400, 400, 1)))
    model.add(MaxPool2D())
    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())

    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.25))

    model.add(Conv2D(128, 3, padding="same", activation="relu"))
    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(Conv2D(32, 3, padding="same", activation="relu"))
    model.add(MaxPool2D())

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    return model


def run_model(path, epochs_number): #Save acc, loss,... in a file to allow interruption
    accuracy = []
    validation_accuracy = []
    loss = []
    validation_loss = []
    try:
        my_model = load_model('/content/drive/MyDrive/Colab Notebooks/' + path)
        print('\033[94m'+'\033[1m' + 'Model Loaded' + '\033[0m')
        my_model.summary()
    except OSError:
        print('\033[91m'+'\033[1m'+'Model', path, 'not found. Creating a new one'+'\033[0m')
        my_model = create_model()
        my_model.summary()
        my_model.compile(loss=tf.keras.losses.BinaryCrossentropy(reduction='sum_over_batch_size'),
                         optimizer=Adam(lr=0.00001), metrics=["accuracy"])

    for i in range(5*epochs_number):
        print('Step (', i+1, '/', epochs_number*5, ')')
        history = my_model.fit(train_images[i%5], train_classes[i%5], epochs=2,
                               validation_data=(validation_images[i%5], validation_classes[i%5]))
        for j in range(2):
            accuracy.append(history.history['accuracy'][j])
            validation_accuracy.append(history.history['val_accuracy'][j])
            loss.append(history.history['loss'][j])
            validation_loss.append(history.history['val_loss'][j])

        my_model.save('/content/drive/MyDrive/Colab Notebooks/' + path)

    return accuracy, validation_accuracy, loss, validation_loss, 10*epochs_number


def test_model(model_path):
    my_model = load_model(model_path)
    mean_accuracy = 0
    mean_loss = 0

    for i in range(5):
        loss, accuracy = my_model.evaluate(test_images[i], test_classes[i])
        mean_accuracy += accuracy/5
        mean_loss += loss/5

    print("Test accuracy:", str(mean_accuracy * 100) + "%")
    print("Test loss:", str(mean_loss * 100) + "%")


def plot_accuracy(accuracy, validation_accuracy, loss, validation_loss, epochs): #Add save as png
    epochs_range = range(epochs)

    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, accuracy, label='Training Accuracy')
    plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, validation_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

train_images =[1,2,3,4,5]
train_classes = [1,2,3,4,5]
validation_images = [1,2,3,4,5]
validation_classes = [1,2,3,4,5]
test_images = [1,2,3,4,5]
test_classes = [1,2,3,4,5]

#for i in range(5):
#     print(i)
#     train_images[i], train_classes[i] = load_data('/content/drive/MyDrive/Colab Notebooks/CSV/train' + str(i + 1) + '.csv')
#     validation_images[i], validation_classes[i] = load_data('/content/drive/MyDrive/Colab Notebooks/CSV/validation' + str(i + 1) + '.csv')
#     test_images[i], test_classes[i] = load_data('/content/drive/MyDrive/Colab Notebooks/CSV/test'+str(i + 1)+'.csv')


#accuracy, validation_accuracy, loss, validation_loss, epochs = run_model('model_02_03_2021.h5', 10)
#print('Acc = ', accuracy)
#print('loss = ', loss)
#print('val_acc = ', validation_accuracy)
#print('val_loss = ', validation_loss)
#plot_accuracy(accuracy, validation_accuracy, loss, validation_loss, epochs)
#test_model('model_01_03_2021.h5')

my_model = create_model()
my_model.summary()

# Test model_01_03_2021.h5 = 96.3644% acc, 0.0946295 loss